In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import heapq
import json
import numpy
import os
import pickle
import random
from tqdm import tqdm
import tensorflow as tf
import uuid

%pylab inline
import matplotlib.pyplot as plt
from PIL import Image

import sys
sys.path.insert(0, "/recsys_data/RecSys/fashion/automl/efficientnetv2")
import effnetv2_model

from data_process import OutfitGen, OutfitGenWithImage

Populating the interactive namespace from numpy and matplotlib


/anaconda/envs/py37_tensorflow/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
import anvil.server
import anvil.media
import anvil.mpl_util

anvil.server.connect("D3BJ3YVOSVMXFCEVTGLVI3XH-5GA6BUC4BEGN43RS")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [4]:
base_dir = "/recsys_data/RecSys/fashion/polyvore-dataset/polyvore_outfits"
data_type = "disjoint" # "nondisjoint"
train_dir = os.path.join(base_dir, data_type)
image_dir = os.path.join(base_dir, "images")
embed_dir = "/recsys_data/RecSys/fashion/polyvore-dataset/precomputed"
model_name = "efficientnet"
checkpoint_filepath = base_dir + "/checkpoint"
image_embedding_file = os.path.join(embed_dir, "effnet_tuned_polyvore.pkl")
data_type = "nondisjoint"
model_type = "rnn"
max_seq_len = 8
item_file = "polyvore_item_metadata.json"

In [5]:
def plot_similar_images(similar_items):
    nrows, ncols = 1, len(similar_items)
    figure, axis = plt.subplots(nrows, ncols, figsize=(20,5))
    items = [tup[1] for tup in similar_items]
    for ii in range(ncols):
        item = items[ii]
        img = plt.imread(os.path.join(image_dir, item + ".jpg"))
        imgplot = axis[ii].imshow(img)
        axis[ii].axis('off')
    plt.show()

def plot_current_outfits(outfits, figsize=(20, 100)):
    nrows = len(outfits)
    ncols = [len(o) for o in outfits][0]
    figure, axis = plt.subplots(nrows, ncols, figsize=figsize)
    for ii in range(nrows):
        items = outfits[ii]
        for jj in range(ncols):
            item = items[jj]
            img = plt.imread(os.path.join(image_dir, item + ".jpg"))
            imgplot = axis[ii][jj].imshow(img)
#             axis[ii][jj].title.set_text(cats[count])
            axis[ii][jj].axis('off')
    plt.show()

In [6]:
with open(image_embedding_file, "rb") as fr:
    image_embedding_dict = pickle.load(fr)
print(f"Loaded {len(image_embedding_dict)} image embeddings")

Loaded 261057 image embeddings


In [7]:
all_files = [image for image in glob.glob(f"{image_dir}/*.jpg")]

In [8]:
eff2_model = tf.keras.models.Sequential(
                [
                    tf.keras.layers.InputLayer(input_shape=[224, 224, 3]),
                    effnetv2_model.get_model("efficientnetv2-b0", include_top=False),
                ]
            )

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


## Search Similar Products

In [9]:
import anvil.media

@anvil.server.callable
def get_similar_products(file):
    with anvil.media.TempFile(file) as filename:
        image = tf.keras.preprocessing.image.load_img(filename)

    image_arr = tf.keras.preprocessing.image.img_to_array(image)
    # image_arr = tf.image.resize(image_arr, (224, 224)).numpy()
    image_arr = tf.image.resize_with_pad(image_arr, target_height=224, target_width=224).numpy()
    image_arr /= 255.0
    image_embed = tf.squeeze(eff2_model(tf.expand_dims(image_arr, 0)))

    q_vec = image_embed #.numpy()
    image_dist = []
    count = 0
    for image, i_vec in tqdm(image_embedding_dict.items()):
        dist = numpy.linalg.norm(q_vec - i_vec.numpy())
        heapq.heappush(image_dist, (dist, image))
        count += 1

    top_images = [heapq.heappop(image_dist) for _ in range(5)]
    top_images = [ anvil.media.from_file(os.path.join(image_dir, item + ".jpg"), 'image/jpeg') for s, item in top_images]
    return top_images


## Outfit Compatibility Score

In [10]:
from focal_loss import BinaryFocalLoss

model = tf.keras.models.load_model('compatibility_nondisjoint_single-transformer_model_8_only_image')
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 8, 1280)]         0         
_________________________________________________________________
dense_39 (Dense)             (None, 8, 64)             81984     
_________________________________________________________________
encoder_1 (Encoder)          (None, 8, 64)             130048    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               328704    
_________________________________________________________________
dense_77 (Dense)             (None, 1)                 257       
Total params: 540,993
Trainable params: 540,993
Non-trainable params: 0
_________________________________________________________________


In [11]:
@anvil.server.callable
def get_compatibility_score(files):
    size = 224, 224
    embed_dim = 1280
    max_len = 8
    outfit_embeddings = []
    count = 0
    zero_elem_image = np.zeros(embed_dim)
    for file in files:
        with anvil.media.TempFile(file) as filename:
            image = tf.keras.preprocessing.image.load_img(filename)
            image_arr = tf.keras.preprocessing.image.img_to_array(image)
            w, h, c = image_arr.shape
            aspect = h*1.0/w
            if aspect > 1.5:
                image_arr = tf.image.resize_with_pad(image_arr, target_height=224, target_width=224).numpy()
            else:
                image_arr = tf.image.resize(image_arr, size).numpy()
            image_arr /= 255.0
            image_embed = tf.squeeze(eff2_model(tf.expand_dims(image_arr, 0)))
            outfit_embeddings.append(image_embed)
            count += 1

    zeros_image = [zero_elem_image for _ in range(max_len - count)]
    outfit_embeddings = np.stack(zeros_image + outfit_embeddings)
    outfit_embeddings = np.expand_dims(outfit_embeddings, 0)
    score = model(outfit_embeddings).numpy()[0][0]
    return score

## Create Outfits

In [12]:
with open(os.path.join(base_dir, item_file), 'r') as fr:
    pv_items = json.load(fr)

In [13]:
category_model = tf.keras.models.load_model(f"finetuned_efficientnet")

In [14]:
id2cat = dict()
cat2items = dict()  # seggregate items by category
for item in pv_items:
    iid, cat = pv_items[item]["category_id"], pv_items[item]["semantic_category"]
    if iid not in id2cat:
        id2cat[iid] = cat
    if cat not in cat2items:
        cat2items[cat] = []
    cat2items[cat].append(item)
    

In [15]:
import pandas as pd

# original logic of creating the labels
# taken from ImageDataGen()
X, y = [], []
for item_id in pv_items:
    X.append(item_id)
    y.append(pv_items[item_id]["category_id"])

X_col = "X"
y_col = "y"
df = pd.DataFrame({X_col: X, y_col: y})
categories = df[y_col].unique()
label_dict = {ii: jj for ii, jj in enumerate(categories)} # reversing

In [16]:
def get_product_category(query):
    image = tf.keras.preprocessing.image.load_img(query)
    image_arr = tf.keras.preprocessing.image.img_to_array(image)
    # image_arr = tf.image.resize(image_arr, (224, 224)).numpy()
    image_arr = tf.image.resize_with_pad(image_arr, target_height=224, target_width=224).numpy()
    image_arr /= 255.0
    pred_score = category_model(tf.expand_dims(image_arr, 0))
    pred_label = np.argmax(pred_score, axis=1)[0]
    pred_label = label_dict[pred_label]  # class number as given in the item description
    return pred_label, id2cat[pred_label]

def get_product_category_2(image_arr):
    pred_score = category_model(tf.expand_dims(image_arr, 0))
    pred_label = np.argmax(pred_score, axis=1)[0]  # index of the class
    pred_label = label_dict[pred_label]  # class number as given in the item description
    return pred_label, id2cat[pred_label]


In [28]:
def return_top_items(query_item, image_embedding_dict, item_dict, ignore=None, max_item=8):
    if type(query_item) is not list:
        query_item = [query_item]
    data_gen = OutfitGenWithImage(embed_dir=embed_dir,
                                  text_embed_file="bert_polyvore.pkl",
                                  batch_size=256,
                                  max_len=max_item,
                                  image_embedding_dim=1280,
                                  query_item=query_item,
                                  image_embedding_dict=image_embedding_dict,
                                  item_description=item_dict,
                                  item_category_dict=cat2items,
                                  ignore_categories=ignore,
                                 )
    pbar = tqdm(range(len(data_gen)))
    current_score = []
    # adds one more item to the list of current items (stored in query_item)
    for ii in pbar:
        x, items = data_gen[ii]
        yhat = model(x)
        for cs, item in zip(yhat, items):
            heapq.heappush(current_score, (1-cs, query_item + [str(item)]))  # it's a min-heap
    return current_score

def filter_outfits(outfits, item_dict, max_len):
    # retain only those combinations where all categories are distinct
    count = 0
    filtered = []
    while count < max_len: 
        outfit = heapq.heappop(outfits)
        items = outfit[1]
        categories = [item_dict[item]['semantic_category'] for item in items]
        if len(set(categories)) == len(categories):
            filtered.append(items)
            count += 1
    return filtered

def create_outfit(query, category, model, image_dict, item_dict, max_item=8, beam_length=10):
    ## TODO: modify for multiple image inputs 
    
    # create new item-id and update embedding_dict and category_dict
    if type(query) is not str:
        item_id = str(uuid.uuid4())
        image_dict[item_id] = query
        item_dict[item_id] = {}
        item_dict[item_id]['semantic_category'] = category
    else:
        item_id = query
    
    # add the first item - only one run
    first_score = return_top_items(item_id, image_dict, item_dict, ignore=[category])
    current_items = filter_outfits(first_score, item_dict, beam_length)
    print(current_items)
    
    for jj in range(2, max_item):
        all_scores = []
        for ii in range(beam_length):
            ignore_categories = [item_dict[item]['semantic_category'] for item in current_items[ii]]
            scores_ii = return_top_items(current_items[ii], image_dict, item_dict, ignore_categories)
            all_scores += scores_ii
        # reconstruct current items - with one new item
        current_items = filter_outfits(all_scores, item_dict, beam_length)
    
    return current_items

def create_outfit_2(query, category, required, model, image_dict, item_dict, beam_length=10):
    # query is always a list of images
    # create new item-id and update embedding_dict and category_dict
    # get only one item
    items = []
    for item, cat in zip(query, category):
        item_id = str(uuid.uuid4())
        image_dict[item_id] = item
        item_dict[item_id] = {}
        item_dict[item_id]['semantic_category'] = cat
        items.append(item_id)
    
    # add the first item - only one run
    if required is not None:
        print(required)
        all_cats = set(list(id2cat.values()))
        category = all_cats.difference(set(required))
        
    first_score = return_top_items(items, image_dict, item_dict, ignore=category)
    if len(category) == len(set(category)):
        # if the original outfit has duplicate categories then the following
        # function will not work
        current_items = filter_outfits(first_score, item_dict, beam_length)
    else:
        count = 0
        current_items = []
        while count < beam_length: 
            outfit = heapq.heappop(first_score)
            items = outfit[1]
            current_items.append(items)
            count += 1
        
    return current_items

['bags', 'shoes', 'jewellery']
['bottoms']
Original 261060 items, reduced to 27673 items


100%|██████████| 109/109 [00:38<00:00,  2.81it/s]


[['cc570db3-fcc3-4fbd-8bf6-7ae12e48e7b8', 'dffd28e3-d3ec-43eb-8e3b-ab455bc41bff', '04537d45-a1ac-49f3-9351-c00c5234b737', '211470046'], ['cc570db3-fcc3-4fbd-8bf6-7ae12e48e7b8', 'dffd28e3-d3ec-43eb-8e3b-ab455bc41bff', '04537d45-a1ac-49f3-9351-c00c5234b737', '33912340'], ['cc570db3-fcc3-4fbd-8bf6-7ae12e48e7b8', 'dffd28e3-d3ec-43eb-8e3b-ab455bc41bff', '04537d45-a1ac-49f3-9351-c00c5234b737', '215035975']]
['bags', 'shoes', 'jewellery']
['tops']
Original 261060 items, reduced to 33001 items


100%|██████████| 129/129 [00:46<00:00,  2.75it/s]


[['60ff7066-66d1-411a-a2a9-54929393514f', 'c5cb4133-0741-49df-b346-c8bcc99b891c', 'ef9ec01b-e2e7-4911-88af-31c06ff9c028', '213513387'], ['60ff7066-66d1-411a-a2a9-54929393514f', 'c5cb4133-0741-49df-b346-c8bcc99b891c', 'ef9ec01b-e2e7-4911-88af-31c06ff9c028', '16896631'], ['60ff7066-66d1-411a-a2a9-54929393514f', 'c5cb4133-0741-49df-b346-c8bcc99b891c', 'ef9ec01b-e2e7-4911-88af-31c06ff9c028', '163204948']]


...

Original 261058 items, reduced to 200946 items


100%|██████████| 785/785 [04:27<00:00,  2.93it/s]


Original 261058 items, reduced to 211940 items


100%|██████████| 828/828 [04:43<00:00,  2.92it/s]


[['c783e4b3-0b26-4f27-9d11-f9b2e57b7c82', '209279359', '135911544'], ['c783e4b3-0b26-4f27-9d11-f9b2e57b7c82', '209279359', '146467964'], ['c783e4b3-0b26-4f27-9d11-f9b2e57b7c82', '209279359', '184520479']]


In [18]:
@anvil.server.callable
def get_outfit(file):
    with anvil.media.TempFile(file) as filename:
        image = tf.keras.preprocessing.image.load_img(filename)

    image_arr = tf.keras.preprocessing.image.img_to_array(image)
    # image_arr = tf.image.resize(image_arr, (224, 224)).numpy()
    image_arr = tf.image.resize_with_pad(image_arr, target_height=224, target_width=224).numpy()
    image_arr /= 255.0
    image_embed = tf.squeeze(eff2_model(tf.expand_dims(image_arr, 0)))
    image_cat_id, image_cat = get_product_category_2(image_arr)
    print(image_cat_id, image_cat)
    outfits = create_outfit(query=image_embed,
                            category=image_cat,
                            model=model,
                            image_dict=image_embedding_dict.copy(), 
                            item_dict=pv_items.copy(), 
                            max_item=3, 
                            beam_length=3)
    print(outfits)
    outfit_images = []
    for prods in outfits:
        res = []
        for item in prods[1:]:
            filepath = os.path.join(image_dir, item + ".jpg")
            res.append(anvil.media.from_file(filepath, 'image/jpeg'))
        outfit_images.append(res)
    return outfit_images


In [27]:
@anvil.server.callable
def get_next_best_product(files, req_categories=None):
    size = 224, 224
    embed_dim = 1280
    max_len = 8
    outfit_embeddings = []
    outfit_categories = []
    for file in files:
        with anvil.media.TempFile(file) as filename:
            image = tf.keras.preprocessing.image.load_img(filename)
            image_arr = tf.keras.preprocessing.image.img_to_array(image)
            w, h, c = image_arr.shape
            aspect = h*1.0/w
            if aspect > 1.5:
                image_arr = tf.image.resize_with_pad(image_arr, target_height=224, target_width=224).numpy()
            else:
                image_arr = tf.image.resize(image_arr, size).numpy()
            image_arr /= 255.0
            image_embed = tf.squeeze(eff2_model(tf.expand_dims(image_arr, 0)))
            image_cat_id, image_cat = get_product_category_2(image_arr)
            outfit_embeddings.append(image_embed)
            outfit_categories.append(image_cat)

    print(outfit_categories)
    if type(req_categories) is str:
        req_categories = [req_categories]
    outfits = create_outfit_2(query=outfit_embeddings,
                              category=outfit_categories,
                              required=req_categories,
                              model=model,
                              image_dict=image_embedding_dict.copy(), 
                              item_dict=pv_items.copy(), 
                              beam_length=3)
    print(outfits)
    outfit_images = [o[-1] for o in outfits]
    res = []
    for item in outfit_images:
        filepath = os.path.join(image_dir, item + ".jpg")
        res.append(anvil.media.from_file(filepath, 'image/jpeg'))
    return res


['bags', 'shoes', 'jewellery']
['bottoms']


In [20]:
# query = "/recsys_data/RecSys/fashion/129978068.jpg"
# image = tf.keras.preprocessing.image.load_img(query)
# image_arr = tf.keras.preprocessing.image.img_to_array(image)
# image_arr = tf.image.resize(image_arr, (224, 224)).numpy()
# # image_arr = tf.image.resize_with_pad(image_arr, target_height=224, target_width=224).numpy()
# image_arr /= 255.0
# image_embed = tf.squeeze(eff2_model(tf.expand_dims(image_arr, 0)))
# image_cat_id, image_cat = get_product_category_2(image_arr)
# print(image_cat_id, image_cat)

In [21]:
# outfits = create_outfit(query=image_embed,
#                         category=image_cat,
#                         model=model,
#                         image_dict=image_embedding_dict.copy(), 
#                         item_dict=pv_items.copy(), max_item=3, beam_length=3)

['bags', 'shoes', 'jewellery', 'jewellery']
Original 261061 items, reduced to 124031 items


100%|██████████| 485/485 [02:52<00:00,  2.81it/s]


[['590012ed-a315-4a28-9962-341a8b73a3e5', '779c0f8a-0c2d-4e72-941c-1476d41e3806', '5c3c2e4f-43cb-4e03-a941-733df1a15f58', '0c9da139-b564-4367-8d19-a9c65505d0ec', '75318199'], ['590012ed-a315-4a28-9962-341a8b73a3e5', '779c0f8a-0c2d-4e72-941c-1476d41e3806', '5c3c2e4f-43cb-4e03-a941-733df1a15f58', '0c9da139-b564-4367-8d19-a9c65505d0ec', '192541085'], ['590012ed-a315-4a28-9962-341a8b73a3e5', '779c0f8a-0c2d-4e72-941c-1476d41e3806', '5c3c2e4f-43cb-4e03-a941-733df1a15f58', '0c9da139-b564-4367-8d19-a9c65505d0ec', '168530467']]
['bags', 'shoes', 'all-body', 'jewellery']
Original 261061 items, reduced to 105553 items


100%|██████████| 413/413 [02:25<00:00,  2.84it/s]


[['d683507e-1e13-4530-957e-5414866dea19', '4601c222-37da-47d6-b448-df29f4a109a7', '818ab690-e8c9-4100-8c34-16b18c9eb845', '49ef6783-cb08-4ae4-bc2c-306f23efdba5', '192541085'], ['d683507e-1e13-4530-957e-5414866dea19', '4601c222-37da-47d6-b448-df29f4a109a7', '818ab690-e8c9-4100-8c34-16b18c9eb845', '49ef6783-cb08-4ae4-bc2c-306f23efdba5', '168530467'], ['d683507e-1e13-4530-957e-5414866dea19', '4601c222-37da-47d6-b448-df29f4a109a7', '818ab690-e8c9-4100-8c34-16b18c9eb845', '49ef6783-cb08-4ae4-bc2c-306f23efdba5', '204438795']]
